In [41]:
import pandas as pd
import sqlite3 as sq
import numpy as np

In [52]:
online_store=pd.read_csv("/Users/Yanaorfin_1/Documents/Academy/E_Commerce/Sample-Superstore2.csv", delimiter=";")
orders=pd.read_csv("/Users/Yanaorfin_1/Documents/Academy/E_Commerce/Sample-Superstore2/Order-Order.csv", delimiter=";")
customers=pd.read_csv("/Users/Yanaorfin_1/Documents/Academy/E_Commerce/Sample-Superstore2/Customer-Customer.csv",delimiter=";")
products=pd.read_csv("/Users/Yanaorfin_1/Documents/Academy/E_Commerce/Sample-Superstore2/Product-Table 1.csv",delimiter=";")
sales=pd.read_csv("/Users/Yanaorfin_1/Documents/Academy/E_Commerce/Sample-Superstore2/Sales-Sales.csv",delimiter=";")
shipment=pd.read_csv("/Users/Yanaorfin_1/Documents/Academy/E_Commerce/Sample-Superstore2/Shipment-Shipment.csv",delimiter=";")

FULL TABLE FOR EDA 

In [64]:
#Additional DataFrame with correct product_id
products2=products.iloc[:,[0,3,4]]

In [74]:
online_store.drop(labels="Row ID",axis=1,inplace=True)

In [70]:
online_store.rename(columns={"Order ID":"Order_ID","Order Date":"Order_Date","Ship Date":"Ship_Date","Ship Mode":"Ship_Mode","Customer ID":"Customer_ID","Customer Name":"Customer_Name","Postal Code":"Postal_Code","Product ID":"Product_ID","Sub-Category":"Sub_Category","Product Name":"Product_Name"},inplace=True)

In [72]:
online_store=pd.merge(online_store,products2,how="left",on=["Product_ID","Product_Name"])

In [ ]:
online_store.drop(labels="Product_ID",axis=1,inplace=True)


REPLACING WRONG Product_ID

In [60]:
sales=pd.merge(sales,products,how="left",on=["Product_ID","Product_Name"])

In [47]:
sales.drop(labels=["Product_ID",'Sub_Category','Category','Product_Name'],axis=1,inplace=True)
products.drop(labels=["Product_ID"],axis=1,inplace=True)


In [48]:

sales.rename(columns={"Product_ID_NEW":"Product_ID"},inplace=True)
products.rename(columns={"Product_ID_NEW":"Product_ID"},inplace=True)

In [49]:
orders = orders.assign(id=range(1,1+len(orders)))

In [50]:
sales=sales.iloc[:,[0,1,6,2,3,4,5]]
products=products.iloc[:,[3,0,1,2]]

In [51]:
sales.head(5)

,Sales_ID,Order_ID,Product_ID,Sales,Discount,Profit,Quantity
0,1,CA-2016-152156,FUR-BO-10001798,261.9600,0.00,41.9136,2
1,2,CA-2016-152156,FUR-CH-10000454,731.9400,0.00,219.5820,3
2,3,CA-2016-138688,OFF-LA-10000240,14.6200,0.00,6.8714,2
3,4,US-2015-108966,FUR-TA-10000577,957.5775,0.45,-383.0310,5
4,5,US-2015-108966,OFF-ST-10000760,22.3680,0.20,2.5164,2


In [ ]:
#orders.to_csv('/Users/Yanaorfin/Documents/Academy/E-Commerce/Order-Order fixed.csv',index=False)

CREATING DATA BASE

In [ ]:
db_conn = sq.connect("/Users/Yanaorfin/Documents/Academy/E_Commerce/onlinestore.db")

In [ ]:
# creating cursor object
cursor=db_conn.cursor()

In [ ]:
orders.head(1)

In [ ]:
# CUSTOMERS 

cursor.execute(
    """
    CREATE TABLE customers (
        Customer_ID VARCHAR (60) NOT NULL PRIMARY KEY, 
        Customer_Name VARCHAR (100) NOT NULL,
        Segment VARCHAR (100)
    );
    """
)



In [ ]:
# PRODUCTS 

cursor.execute(
    """
    CREATE TABLE products (
        Product_id VARCHAR (60) NOT NULL PRIMARY KEY,
        Category VARCHAR (100),
        Sub_Category VARCHAR (100),
        Product_Name VARCHAR (200)
    );
    """
)

# ORDERS 
cursor.execute(
    """
    CREATE TABLE orders (
        Order_Id VARCHAR (60) PRIMARY KEY NOT NULL,
        Order_Date VARCHAR (20) NOT NULL,
        Customer_ID VARCHAR (60),
        FOREIGN KEY (Customer_ID) REFERENCES customers (Customer_ID)
    );
    """ 
)

#SHIPMENT

cursor.execute(
    """
    CREATE TABLE shipment (
        Shipment_id INTEGER PRIMARY KEY NOT NULL,
        Order_Id VARCHAR (60) NOT NULL,
        Ship_Date VARCHAR (20) NOT NULL, 
        Ship_Mode VARCHAR (60),
        Country VARCHAR (60),
        City VARCHAR (60),
        State VARCHAR (60),
        Postal_Code VARCHAR (60),
        Region VARCHAR (60),
        FOREIGN KEY (Order_Id) REFERENCES orders (Order_Id) 
    );
    """
)

# SALES 

cursor.execute(
    """
    CREATE TABLE sales (
        Sales_Id INTEGER PRIMARY KEY NOT NULL,
        Order_Id VARCHAR (60) NOT NULL,
        Product_ID VARCHAR (60),
        Sales REAL,
        Discount REAL,
        Profit REAL,
        Quantity INTEGER,
        FOREIGN KEY (Order_Id) REFERENCES orders (Order_Id),
        FOREIGN KEY (Product_ID) REFERENCES products (Product_ID)

);
"""
) 

In [ ]:

sales.to_sql('sales', db_conn, if_exists='append', index=False)
orders.to_sql('orders', db_conn, if_exists='append', index=False)
customers.to_sql('customers', db_conn, if_exists='append', index=False)


In [ ]:
products.to_sql('products', db_conn, if_exists='append', index=False)
shipment.to_sql('shipment',db_conn, if_exists='append', index=False)